### Step1: Load dataset

In [ ]:
import pandas as pd
import numpy as np

# Path to extracted TXT file
file_path = 'household_power_consumption.txt'

# Load with ; separator, handle '?' as NaN
data = pd.read_csv(file_path, sep=';', na_values='?')

# Combine Date and Time to datetime
data['datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%d/%m/%Y %H:%M:%S')

# Numerical timestamp: seconds since the earliest date
min_dt = data['datetime'].min()
data['timestamp'] = (data['datetime'] - min_dt).dt.total_seconds()

# Relevant columns (drop Date/Time/datetime, keep numerics)
cols = ['timestamp', 'Global_active_power', 'Global_reactive_power', 'Voltage', 
        'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
data = data[cols].dropna()  # ~1.25% missing, drop for simplicity
full_data_size = len(data)

print(f"Dataset loaded: {data.shape[0]} rows")
print(f"First 5 data: {data.head()}")
print(f"Data size: {full_data_size}")


Dataset loaded: 2049280 rows
First 5 data:    timestamp  Global_active_power  Global_reactive_power  Voltage  \
0        0.0                4.216                  0.418   234.84   
1       60.0                5.360                  0.436   233.63   
2      120.0                5.374                  0.498   233.29   
3      180.0                5.388                  0.502   233.74   
4      240.0                3.666                  0.528   235.68   

   Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
0              18.4             0.0             1.0            17.0  
1              23.0             0.0             1.0            16.0  
2              23.0             0.0             2.0            17.0  
3              23.0             0.0             1.0            17.0  
4              15.8             0.0             1.0            17.0  
Data size: 2049280


### Step2: Create a Small Offline Sample

In [2]:
sample_size = int(0.001 * len(data))  # ~2000 rows
sample = data.sample(n=sample_size, random_state=42).copy()
print(f"Sample created: {sample.shape[0]} rows")
print(sample.head())

Sample created: 2049 rows
          timestamp  Global_active_power  Global_reactive_power  Voltage  \
1030580  61834800.0                1.502                  0.074   240.17   
1815       108900.0                0.374                  0.264   245.50   
1295977  77758620.0                0.620                  0.300   239.85   
206669   12400140.0                0.280                  0.200   235.72   
1048893  62933580.0                1.372                  0.054   243.95   

         Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
1030580               6.4             0.0             0.0            18.0  
1815                  1.8             0.0             2.0             0.0  
1295977               3.0             0.0             1.0             1.0  
206669                1.4             0.0             0.0             0.0  
1048893               5.6             0.0             0.0            18.0  


### Step 3: Generate a Historical Query Log
直接抓query因為dimension很大的問題，result有可能會是0，因此先將範圍大概縮在中間50%。

In [3]:
from funcs import generate_random_query, exact_sum, sample_sum

# Define dimensions (7D) and their min/max
dimensions = ['timestamp', 'Global_reactive_power', 'Voltage', 'Global_intensity', 
              'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']

q = generate_random_query(data, dimensions, test = True)
print(q)

dim: timestamp, min value: 0.0, max value: 124515480.0
dim: Global_reactive_power, min value: 0.0, max value: 1.39
dim: Voltage, min value: 223.2, max value: 254.15
dim: Global_intensity, min value: 0.2, max value: 48.4
dim: Sub_metering_1, min value: 0.0, max value: 88.0
dim: Sub_metering_2, min value: 0.0, max value: 80.0
dim: Sub_metering_3, min value: 0.0, max value: 31.0
{'timestamp': (2594830.0139828413, 115491449.07118052), 'Global_reactive_power': (0.23966607177486599, 1.1370852710695831), 'Voltage': (225.12716189494918, 247.86576800729415), 'Global_intensity': (4.96397750296437, 36.96420798178017), 'Sub_metering_1': (14.096431920137086, 85.49236232258518), 'Sub_metering_2': (5.486723722658424, 66.84763094086101), 'Sub_metering_3': (3.374133516707752, 25.790545769926748)}


Functions for exact sum, sample sum.

建立query log

In [6]:
# Aggregate column
agg_col = 'Global_active_power'

# 設定一個avg exact讓threshold有依據
print("Generate avg_exact by temp queries...")
temp_query_log = []
num_queries = 2000
attempts = 0
max_attempts = 10000

while len(temp_query_log) < num_queries and attempts < max_attempts:
    q = generate_random_query(data, dimensions)
    exact_result = exact_sum(agg_col, q, data)
    if exact_result > 1.0:  # 會有result為0的情況，設定threshold
        estimate = sample_sum(agg_col, q, sample, full_data_size)
        error = exact_result - estimate
        temp_query_log.append({'query': q, 'exact': exact_result, 
                          'estimate': estimate, 'error': error})
    attempts += 1

avg_exact = np.mean([temp_query['exact'] for temp_query in temp_query_log])
print(f"Generated {len(temp_query_log)} temp queries in {attempts} attempts")
print(f"Average exact sum: {avg_exact=}")

# For training: 2000 queries (as in paper)
query_log = []
print("Generating 2000 queries...")
while len(query_log) < num_queries and attempts < max_attempts:
    q = generate_random_query(data, dimensions)
    exact_result = exact_sum(agg_col, q, data)
    
    if exact_result > 0.01 * avg_exact: 
        estimate = sample_sum(agg_col, q, sample, full_data_size)
        error = exact_result - estimate
        query_log.append({'query': q, 'exact': exact_result, 
                          'estimate': estimate, 'error': error})
    attempts += 1

print(f"Generated {len(query_log)} queries in {attempts} attempts")

Generate avg_exact by temp queries...
Generated 2000 temp queries in 2000 attempts
Average exact sum: avg_exact=10230.512895
Generating 2000 queries...
Generated 2000 queries in 4000 attempts


In [30]:
query_log[0]

{'query': {'timestamp': (317889.14719108393, 95484735.34709866),
  'Global_reactive_power': (0.08176741717106216, 1.2001717279694588),
  'Voltage': (224.99723593786194, 253.5116929519517),
  'Global_intensity': (10.255683845754504, 45.795909137749476),
  'Sub_metering_1': (11.41996008670598, 68.08800969980449),
  'Sub_metering_2': (0.613595062827188, 69.98365714218588),
  'Sub_metering_3': (3.5054635130669345, 27.912309334612747)},
 'exact': 42442.814000000006,
 'estimate': 38571.270122010734,
 'error': 3871.543877989272}

### Step 4: Compute Sampling-Based Estimates and Errors for the Query Log

Diversification: training data

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
import random

scaler_div = StandardScaler()

def diversify_query_log(query_log, k):
    """
    Diversify the query log using greedy max-min (paper Section 5.2).
    Features: flattened ranges + error.
    Returns a subset of k diverse entries.
    """
    # Prepare features: flatten query bounds + error
    features = []
    for entry in query_log:
        vec = []
        for dim in dimensions:
            lower, upper = entry['query'][dim]
            vec.extend([lower, upper])
        vec.append(entry['error'])
        features.append(vec)
    
    features = np.array(features)
    features_norm = scaler_div.fit_transform(features)
    
    # Greedy max-min selection
    selected_indices = [random.randint(0, len(features)-1)]  # Start with random
    selected_features = features_norm[selected_indices]
    
    while len(selected_indices) < k:
        # Distances from unselected to current selected set
        dists = euclidean_distances(selected_features, features_norm)
        min_dists = dists.min(axis=0)  # Min dist to any selected
        # Pick the one with max min-dist (most diverse)
        next_idx = np.argmax(min_dists)
        selected_indices.append(next_idx)
        selected_features = features_norm[selected_indices]
    
    diversified_log = [query_log[i] for i in selected_indices]
    print(f"Diversified query log: selected {len(diversified_log)} / {len(query_log)} entries")
    return diversified_log

diversified_log = diversify_query_log(query_log, k=800) 

Diversified query log: selected 1000 / 2000 entries


Generate testing data.

In [ ]:
print("Generate testing...")
temp_query_log = []
num_queries = 100
attempts = 0
max_attempts = 1000

while len(temp_query_log) < num_queries and attempts < max_attempts:
    q = generate_random_query(data, dimensions)
    exact_result = exact_sum(agg_col, q, data)
    if exact_result > 0.01 * avg_exact:  # 會有result為0的情況，設定threshold
        estimate = sample_sum(agg_col, q, sample, full_data_size)
        error = exact_result - estimate
        temp_query_log.append({'query': q, 'exact': exact_result, 
                          'estimate': estimate, 'error': error})
    attempts += 1

avg_exact = np.mean([temp_query['exact'] for temp_query in temp_query_log])
print(f"Generated {len(temp_query_log)} testing data in {attempts} attempts")

Load data if needed

In [ ]:
# import pickle

# # 讀取資料
# with open('diversify_log.pkl', 'rb') as f:
#     diversified_log = pickle.load(f)

### Step 5: Train the Error Prediction Model

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Prepare features (flatten: lower/upper per dim) and targets (errors)
X = []
y = []
# for entry in query_log:
for entry in diversified_log:

    vec = []
    for dim in dimensions:
        lower, upper = entry['query'][dim]
        vec.extend([lower, upper])
    X.append(vec)
    y.append(entry['error'])

X = np.array(X)
y = np.array(y)

# Normalize
X_scaled = scaler_div.fit_transform(X)

# Train (80/20 split for validation)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
# adjust max_depth
model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
model.fit(X_train, y_train)

print(f"Model trained.")
# print(f"Test MSE: {np.mean((model.predict(X_test) - y_test)**2)}")

Model trained.


### Step 6: Estimate a New Query

In [33]:
# Example new 7D query (adjust ranges to sensible values based on data mins/maxes)
new_query = {
    'timestamp': (0, 1e8),  # e.g., first ~few months in seconds
    'Global_reactive_power': (0.0, 0.5),
    'Voltage': (220, 250),
    'Global_intensity': (0, 20),
    'Sub_metering_1': (0, 10),
    'Sub_metering_2': (0, 5),
    'Sub_metering_3': (0, 15)
}

# Flatten and scale
new_vec = []
for dim in dimensions:
    lower, upper = new_query[dim]
    new_vec.extend([lower, upper])
new_vec = np.array([new_vec])
new_scaled = scaler_div.transform(new_vec)

# Predict error
predicted_error = model.predict(new_scaled)[0]

# Find error-similar historical query (closest error)
min_diff = float('inf')
opt_entry = None
for entry in diversified_log:
    error_diff = abs(entry['error'] - predicted_error)
    if error_diff < min_diff:
        min_diff = error_diff
        opt_entry = entry

# Compute final estimate
sample_new = sample_sum(new_query, sample, full_data_size)
sample_opt = opt_entry['estimate']
final_estimate = opt_entry['exact'] + (sample_new - sample_opt)

print(f"LAQP estimate: {final_estimate:.2f}")
# Compute exact for the same query (for debugging/small queries)
exact = exact_sum(new_query, data)
print(f"Exact sum: {exact:.2f}")
print(f"Relative error: {abs(final_estimate - exact) / exact:.4f}")

# Also see how many rows match
# mask = np.ones(len(data), dtype=bool)
# for dim, (l, u) in new_query.items():
#     mask &= (data[dim] >= l) & (data[dim] <= u)
# matched_rows = mask.sum()
# print(f"Query matches {matched_rows:,} rows ({matched_rows / len(data):.1%} of dataset)")

LAQP estimate: 555401.61
Exact sum: 560494.03
Relative error: 0.0091


In [34]:
def laqp_estimate_with_details(query):
    # Flatten and predict error (same as before)
    vec = []
    for dim in dimensions:
        l, u = query[dim]
        vec.extend([l, u])
    vec = np.array([vec])
    scaled = scaler_div.transform(vec)
    pred_error = model.predict(scaled)[0]
    
    # Find the most error-similar historical query
    best_index = -1
    best_error_diff = float('inf')
    best_entry = None

    for idx, entry in enumerate(diversified_log):
        error_diff = abs(entry['error'] - pred_error)
        if error_diff < best_error_diff:
            best_error_diff = error_diff
            best_index = idx
            best_entry = entry
    
    # Compute estimates
    sample_new = sample_sum(query, sample, full_data_size)
    sample_opt = best_entry['estimate']
    final_est = best_entry['exact'] + (sample_new - sample_opt)
    
    print(f"Selected optimal query index: {best_index} (out of  {len(diversified_log)})")
    print(f"Predicted error for new query: {pred_error:.2f}")
    print(f"Chosen historical query error: {best_entry['error']:.2f} (diff: {best_error_diff:.2f})")
    print(f"Exact result of chosen query: {best_entry['exact']:.2f}")
    print("Predicate ranges of chosen query:")
    for dim, (l, u) in best_entry['query'].items():
        print(f"  {dim}: [{l:.2f}, {u:.2f}]")
    print(f"\nFinal LAQP estimate: {final_est:.2f}")    

    return final_est, best_index, best_entry

# Use it
# estimate, opt_idx, opt_entry = laqp_estimate_with_details(new_query)

### Step 7: Evaluate and Extend
Basic Evaluation: Measure Accuracy on Test Queries

Optimization


In [35]:
def range_distance(q1, q2):
    """Euclidean distance on flattened predicate bounds (for range-similarity)."""
    vec1 = []
    vec2 = []
    for dim in dimensions:
        l1, u1 = q1[dim]
        l2, u2 = q2[dim]
        vec1.extend([l1, u1])
        vec2.extend([l2, u2])
    return np.linalg.norm(np.array(vec1) - np.array(vec2))

In [36]:
from scipy.optimize import minimize_scalar

def optimize_alpha(val_queries, model, sample, full_data_size, bounds=(0,1)):
    """
    Tune alpha for hybrid similarity (paper Section 5.3).
    val_queries: List of {'query': dict, 'exact': float} for tuning.
    Returns best alpha that minimizes average relative error on val set.
    """
    def objective(alpha):
        errors = []
        for vq in val_queries:
            query = vq['query']
            exact = vq['exact']
            
            # Predict error
            vec = [query[dim][i] for dim in dimensions for i in range(2)]
            vec = np.array([vec])
            scaled = scaler_div.transform(vec)
            pred_error = model.predict(scaled)[0]
            
            # Find best entry with hybrid similarity
            best_entry = min(diversified_log, key=lambda e: 
                alpha * abs(e['error'] - pred_error) + 
                (1 - alpha) * range_distance(query, e['query']))
            
            # LAQP estimate
            sample_new = sample_sum(query, sample, full_data_size)
            sample_opt = best_entry['estimate']
            laqp_est = best_entry['exact'] + (sample_new - sample_opt)
            
            # Relative error
            rel_err = abs(laqp_est - exact) / (exact + 1e-6)
            errors.append(rel_err)
        
        return np.mean(errors)
    
    # Optimize alpha
    res = minimize_scalar(objective, bounds=bounds, method='bounded')
    best_alpha = res.x
    print(f"Optimized alpha: {best_alpha:.3f} (MSE on val: {res.fun:.4f})")
    return best_alpha

In [ ]:
# Generate 100 separate test queries
test_queries = []
num_test = 100
attempts = 0
max_attempts = 1000

print("Generating 100 test queries...")
while len(test_queries) < num_test and attempts < max_attempts:
    q = generate_random_query(data, dimensions)
    exact_result = exact_sum(q, data)
    if exact_result > 0.01 * avg_exact:
        test_queries.append({'query': q, 'exact': exact_result})

print(f"Generated {len(test_queries)} test queries")

best_alpha = optimize_alpha(test_queries, model, sample, full_data_size)

# Evaluation lists
laqp_rel_errors = []
sampling_rel_errors = []
laqp_abs_errors = []
sampling_abs_errors = []

# Threshold for filtering tiny queries (adjustable)
# Good starting values: 1000 or 0.01 * average exact sum
avg_exact = np.mean([tq['exact'] for tq in test_queries])
min_exact_threshold = max(1000.0, 0.01 * avg_exact)  # at least 1000 or 1% of avg

print(f"Using minimum exact threshold: {min_exact_threshold:.2f} "
      f"(avg exact = {avg_exact:.2f})")

Generating 100 test queries...
Generated 100 test queries
Optimized alpha: 0.618 (MSE on val: 0.5282)
Using minimum exact threshold: 1000.00 (avg exact = 10006.51)


In [39]:
filtered_count = 0

for tq in test_queries:
    query = tq['query']
    exact = tq['exact']
    
    if exact < min_exact_threshold:
        continue  # skip tiny queries that distort relative error
    filtered_count += 1
    
    # --- Pure Sampling Estimate ---
    sample_est = sample_sum(query, sample, full_data_size)
    sampling_abs = abs(sample_est - exact)
    sampling_rel = sampling_abs / exact
    
    sampling_abs_errors.append(sampling_abs)
    sampling_rel_errors.append(sampling_rel)
    
    # --- LAQP Estimate ---
    # Flatten and predict error
    vec = []
    for dim in dimensions:
        l, u = query[dim]
        vec.extend([l, u])
    vec = np.array([vec])
    scaled = scaler_div.transform(vec)
    predicted_error = model.predict(scaled)[0]
    
    # Hybrid selection
    best_entry = min(diversified_log, key=lambda e: 
        best_alpha * abs(e['error'] - predicted_error) + 
        (1 - best_alpha) * range_distance(query, e['query']))
    
    # Compute estimates
    sample_new = sample_sum(query, sample, full_data_size)
    sample_opt = best_entry['estimate']
    laqp_est = best_entry['exact'] + (sample_new - sample_opt)
    
    laqp_abs = abs(laqp_est - exact)
    laqp_rel = laqp_abs / exact
    
    laqp_abs_errors.append(laqp_abs)
    laqp_rel_errors.append(laqp_rel)

# ========================
# Results
# ========================
print("\n" + "="*50)
print("EVALUATION RESULTS")
print("="*50)
print(f"Evaluated on {filtered_count} / {len(test_queries)} queries "
      f"(excluded {len(test_queries)-filtered_count} tiny ones)")

if len(laqp_rel_errors) > 0:
    print(f"\nLAQP    Average Relative Error (ARE): {np.mean(laqp_rel_errors):.4f}")
    print(f"LAQP    Median Relative Error:       {np.median(laqp_rel_errors):.4f}")
    print(f"LAQP    Mean Absolute Error (MAE):     {np.mean(laqp_abs_errors):.2f}")
    
    print(f"\nSampling ARE:                       {np.mean(sampling_rel_errors):.4f}")
    print(f"Sampling Median Relative Error:     {np.median(sampling_rel_errors):.4f}")
    print(f"Sampling MAE:                       {np.mean(sampling_abs_errors):.2f}")
    
    improvement = np.mean(sampling_rel_errors) / np.mean(laqp_rel_errors)
    print(f"\nImprovement (Sampling ARE / LAQP ARE): {improvement:.2f}x")
else:
    print("No queries passed the threshold — try lowering min_exact_threshold")


EVALUATION RESULTS
Evaluated on 100 / 100 queries (excluded 0 tiny ones)

LAQP    Average Relative Error (ARE): 0.5282
LAQP    Median Relative Error:       0.3287
LAQP    Mean Absolute Error (MAE):     4319.04

Sampling ARE:                       0.6656
Sampling Median Relative Error:     0.6502
Sampling MAE:                       5472.43

Improvement (Sampling ARE / LAQP ARE): 1.26x


### Store data if needed

In [ ]:
import pickle

# 儲存資料 (原本的 data_list)
with open('data_storage.pkl', 'wb') as f:
    pickle.dump(diversified_log, f)